# 量子機械学習


Kifumi Numata (Aug 18, 2023)

In [ ]:
import numpy as np

# 描画のため
import matplotlib.pyplot as plt

# Scikit-learnのインポート(Python用の機械学習ライブラリー)
from sklearn import datasets
from sklearn.model_selection import train_test_split # データ分割
from sklearn.svm import SVC # SVM Classification(SVM分類)
from sklearn.decomposition import PCA # Principal component analysis(主成分分析)
from sklearn.preprocessing import StandardScaler, MinMaxScaler # 標準化、正規化のスケール変換

# Qiskitのインポート
from qiskit import Aer, execute
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler

## データを用意

ここでは，手書き数字画像のデータセット(MNISTデータセット)から0と1のサブセットを扱います。

In [ ]:
# 数字データセットから2クラスのデータ(0と1)を読み込み
digits = datasets.load_digits(n_class=2)   

# 読み込んだ画像の最初の100枚をプロット
fig, axes = plt.subplots(10, 10, figsize=(15, 15), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.5, wspace=0.5))
for i, ax in enumerate(axes.flat):
    ax.set_axis_off()
    ax.imshow(digits.images[i], cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title(digits.target[i])

In [ ]:
print(digits.images[0])

このデータセットには、合計360個のデータが含まれています。各データポイントは、8×8の数字の画像で、配列になっていて、各要素は0（白）から16（黒）までの整数です。古典的な分類アルゴリズムの際と同様に、データセットを学習用（40個）とテスト用（10個）のサンプルに分割し、正規化する必要があります。このデータセットを量子分類アルゴリズムに用いるために、範囲を-1から1の間にスケーリングし、次元を使用する量子ビット数（今回は4）に縮小します。

In [ ]:
# データセットの分割
sample_train, sample_test, labels_train, labels_test = train_test_split(
     digits.data, digits.target, test_size=0.4, random_state=22)

# 次元削除
n_dim = 4
pca = PCA(n_components=n_dim).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# 正規化
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# スケーリング
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# 学習用40個とテスト用10個を選択
train_size = 40
sample_train = sample_train[:train_size]
labels_train = labels_train[:train_size]

test_size = 10
sample_test = sample_test[:test_size]
labels_test = labels_test[:test_size]

In [ ]:
# 一つ目のデータをそれぞれ表示
print(sample_train[0], labels_train[0])
print(sample_test[0], labels_test[0])

In [ ]:
# 学習用データのラベルを表示
print(labels_train)

## データ符号化(エンコード)

この古典データを、量子特徴量マップ(ZZFeatureMap)を用いて量子状態空間にエンコードしていきます。

In [ ]:
# 4特徴量、深さ(繰り返し数)1のZZFeatureMap
zz_map = ZZFeatureMap(feature_dimension=4, reps=1, entanglement='linear', insert_barriers=True)
zz_map.decompose().draw('mpl')

### 量子カーネルの計算

ZZFeatureMapを使って、学習データの0個目と1個目のデータについて、量子カーネルを計算する量子回路を作成し、実際に計算してみます。この値は、二つの量子状態のFidelity(忠実度)ともいいます。

In [ ]:
print(sample_train[0])
print(sample_train[1])

In [ ]:
zz_map = ZZFeatureMap(feature_dimension=4, reps=1, entanglement='linear')
qc_1 = zz_map.bind_parameters(sample_train[0])
qc_2 = zz_map.bind_parameters(sample_train[1])
fidelity_circuit = qc_1.copy()
fidelity_circuit.append(qc_2.inverse().decompose(), range(fidelity_circuit.num_qubits))
fidelity_circuit.measure_all()
fidelity_circuit.decompose().draw('mpl')

各回転ゲートのパラメーター値は少し読みにくいですが、回路が対称になっていることが分かると思います。左半分は訓練データsample_train[0]が、右半分は訓練データsample_train[1]がコード化されています。

例として、上記の量子カーネルを測定し、ゼロ状態のカウント数の割合として、カーネル行列の要素（訓練データの0個目と1個目のデータについて）を計算します。

In [ ]:
# Use Aer's qasm_simulator
from qiskit import Aer
backend_sim = Aer.get_backend('qasm_simulator')

from qiskit import transpile
job_sim = backend_sim.run(transpile(fidelity_circuit, backend_sim), shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(fidelity_circuit)

In [ ]:
counts['0000']/sum(counts.values())

### 量子カーネル計算

このプロセスを，学習データサンプルのペアごとに繰り返して学習カーネル行列を埋め，学習データサンプルとテストデータサンプルの間で繰り返してテストカーネル行列を埋めていきます。
Qiskit Machine Learning の ``FidelityQuantumKernel`` クラスをセットアップし、 `ZZFeatureMap` を用いて量子カーネル行列を計算します。ここでは、 ``Sampler`` primitiveと、状態間のオーバーラップ(二つの状態の重なり具合)を計算する ``ComputeUncompute`` fidelityを使用します。

In [ ]:
zz_map = ZZFeatureMap(feature_dimension=4, reps=1, entanglement='linear')
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
zz_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=zz_map)

訓練データとテストデータについて、カーネル行列を計算します。

In [ ]:
# 学習データ同士のカーネル行列計算(40x40)
matrix_train = zz_kernel.evaluate(x_vec=sample_train)

# 学習データとテストデータとのカーネル行列計算(40x10)
matrix_test = zz_kernel.evaluate(x_vec=sample_test, y_vec=sample_train)

# カーネル行列の表示
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("testing kernel matrix")
plt.show()

## 学習
古典SVM分類器`scikit-learn`の`svc`アルゴリズムを使って学習し、テストデータで学習率を見てみます。

In [ ]:
zz_svc = SVC(kernel='precomputed') #事前に計算したカーネルを使用
zz_svc.fit(matrix_train, labels_train)  #学習データ同士のカーネルとラベルを使って学習

#学習データとテストデータとのカーネル行列を使ってテストデータのラベルを予測
label_predict = zz_svc.predict(matrix_test)
print(label_predict, labels_test)

In [ ]:
#学習率を表示
zz_score = zz_svc.score(matrix_test, labels_test) 
print(f'学習率は {zz_score}')

テストデータポイントを正しく分類できていることがわかります。

# 洋服画像データの分類


ここで扱うデータは，MNISTデータセットの亜種である[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist/blob/master/README.ja.md)という洋服画像データセットのサブセットです。


<center><div><img src="fashion-mnist-sprite.png" width="640" /></div></center>

Image source:[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist/blob/master/README.ja.md)

以下のラベルの画像について分類します。

- label 2: プルオーバー
- label 3: ドレス

まずデータセットを読み込んで，クラスごとに1枚ずつ画像を表示してみます。

In [ ]:
# データのロード
DATA_PATH = 'fashion.npz'
data = np.load(DATA_PATH)

sample_train = data['sample_train']
labels_train = data['labels_train']
sample_test = data['sample_test']

# データセットの分割
sample_train, sample_test, labels_train, labels_test = train_test_split(
    sample_train, labels_train, test_size=0.2, random_state=42)

# データの表示
fig = plt.figure()
LABELS = [2,3]
num_labels = len(LABELS)
for i in range(num_labels):
    ax = fig.add_subplot(2, num_labels, i+1)
    img = sample_train[labels_train==LABELS[i]][0].reshape((28, 28))
    ax.imshow(img, cmap="Greys")

次に，以下のデータセットの前処理をします

- 主成分分析(PCA)による次元圧縮
- 正規化
- スケーリング
- 学習用（80個）とテスト用（20個）のサンプルを選択


In [ ]:
# 次元削除
N_DIM = 4
pca = PCA(n_components=N_DIM).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# 正規化
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# スケーリング
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# 選択
train_size = 80
sample_train = sample_train[:train_size]
labels_train = labels_train[:train_size]

test_size = 20
sample_test = sample_test[:test_size]
labels_test = labels_test[:test_size]

In [ ]:
# 一つ目のデータをそれぞれ表示
print(sample_train[0], labels_train[0])
print(sample_test[0], labels_test[0])

## 演習

この洋服画像データについて、特徴量マップ(ZZFeatureMap)と量子カーネル(QuantumKernelクラス)を使ってカーネル行列を計算し、古典のSVMを使って学習してみます。学習率を確認してみましょう。

In [ ]:
zz_map = # コードを入れてください
zz_map.decompose().draw('mpl')

訓練データの0個目と1個目のデータについて、量子カーネルを計算する量子回路を作成し、実際に計算してみます。

In [ ]:
print(sample_train[0])
print(sample_train[1])

In [ ]:
zz_map = ZZFeatureMap(feature_dimension=4, reps=1, entanglement='linear')
qc_1 = zz_map.bind_parameters(sample_train[0])
qc_2 = zz_map.bind_parameters(sample_train[1])
fidelity_circuit = qc_1.copy()
fidelity_circuit.append(qc_2.inverse().decompose(), range(fidelity_circuit.num_qubits))
fidelity_circuit.measure_all()
fidelity_circuit.decompose().draw('mpl')

In [ ]:
# Use Aer's qasm_simulator
from qiskit import Aer
backend_sim = Aer.get_backend('qasm_simulator')

from qiskit import transpile
job_sim = backend_sim.run(transpile(fidelity_circuit, backend_sim), shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(fidelity_circuit)

In [ ]:
counts['0000']/sum(counts.values())

訓練データとテストデータについて、QuantumKernelクラスを使ってカーネル行列を計算します。

In [ ]:
sampler = Sampler()
fidelity = # コードを入れてください
zz_kernel = # コードを入れてください

In [ ]:
matrix_train = # コードを入れてください
matrix_test = # コードを入れてください

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("testing kernel matrix")
plt.show()

古典SVM分類器`scikit-learn`の`svc`アルゴリズムを使って学習し、テストデータで学習率を見てみます。

In [ ]:
zz_svc = # コードを入れてください
zz_svc.fit(matrix_train, labels_train) #学習データ同士のカーネルとラベルを使って学習

#学習データとテストデータとのカーネル行列を使ってテストデータのラベルを予測
label_predict = # コードを入れてください
print(label_predict, labels_test)

In [ ]:
#学習率を表示
zz_score = zz_svc.score(matrix_test, labels_test) 
print(f'学習率は {zz_score}')

## 時間の余った方向け

データセットunknown_dataが プルオーバー（ラベル2）、またはドレス（ラベル3）、どちらのデータセットであるかを学習した量子カーネル行列を使って、SVMで判別してください。データunknown_dataは、10個の同じラベルのデータセットを次元削除、正規化、スケーリングしたものです。

In [ ]:
unknown_data =[[-0.7506181229786677, 0.3639757115940703, -0.40259035779756747, 0.13156494990059703],
               [0.6876543623856463, -0.14395585615940995, -1.0, 0.18833132593453705],
               [0.1597345117739556, 0.037029365985354834, -0.74170020071629, -0.35871272660416686],
               [0.12415150114857995, 0.15427446808127593, 0.08476843970362899, -0.6089690897810438],
               [0.5488865570214291, 0.17113499449186953, -0.06577365245877217, -0.17285785841016588],
               [-0.5332555688379076, 0.5395437640172006, -0.3128336939755401, -0.6498356127249841],
               [0.8161223476490759, -0.20027533959873733, -0.21196321310952376, -0.1737145763437467],
               [0.4625041945567496, 0.1150438935335493, 0.02083253784257244, -0.20607780130264472],
               [0.6477170524202525, -0.3435526101878924, -0.5108689103689058, -0.3800636730989028],
               [-0.09984118916814182, 0.4890014133606272, -0.14802829167372678, -0.18753891291420458]]

In [ ]:
# 学習データとunknown_dataデータとのカーネル行列(80x10)を計算します


In [ ]:
# 学習データとunknown_dataデータとのカーネル行列を使って
# unknown_dataデータのラベルを予測します


In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table